<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/et_whisper_small_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-eutn5x6i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-eutn5x6i
  Resolved https://github.com/huggingface/transformers to commit 238d2e3c44366aba9dc5c770c95475765a6725cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269614 sha256=397a894686286f22610e9438208bab2f90be3f46a9152acfa6a3c8e340a1249b
  Stored in directory: /tmp/pip-ephem-wheel-cache-spqii9is/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [60]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "et", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [61]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 51
})

In [62]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [103]:
model = whisper.load_model("small")

In [104]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [105]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [106]:
text

,0
0,esiteks on oluline et euroopa liit tuleks välj...
1,härra eesistuja juba süüria kodusõda tekitas s...
2,ja lõpetuseks tuleb kehtestada rahvusvahelised...
3,eelnimetatud rahvusvahelise õiguse rikkumised ...
4,mulle meeldib väga punkt üks kus öeldakse et i...
5,ja ma loeksin ette hea meelega resolutsiooni p...
6,siin on suur tähtsus mitteformaalse ja informa...
7,elu kinnitas taas kord et ida eurooplaste kann...
8,euroopa liidule ja eriti rahva poolt otse vali...
9,õigusriiki ja demokraatiat tugevdavaid reforme...


In [107]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [108]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,esiteks on oluline et euroopa liit tuleks välj...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005340576171875, -0.004791259765625, -0.00...",16000
1,male,härra eesistuja juba süüria kodusõda tekitas s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001129150390625, -0.001190185546875, -0.00...",16000
2,male,ja lõpetuseks tuleb kehtestada rahvusvahelised...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.014007568359375, 0.0140380859375, 0.0124206...",16000
3,female,eelnimetatud rahvusvahelise õiguse rikkumised ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001190185546875, -0.00128173828125, -0.000...",16000
4,female,mulle meeldib väga punkt üks kus öeldakse et i...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.07452392578125, 0.05059814453125, 0.0300292...",16000
5,female,ja ma loeksin ette hea meelega resolutsiooni p...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.038238525390625, 0.0269775390625, 0.0199890...",16000
6,female,siin on suur tähtsus mitteformaalse ja informa...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001983642578125, -0.0028076171875, -0.00152...",16000
7,female,elu kinnitas taas kord et ida eurooplaste kann...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.017974853515625, -0.020294189453125, -0.018...",16000
8,female,euroopa liidule ja eriti rahva poolt otse vali...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005035400390625, 0.001007080078125, 0.0037...",16000
9,male,õigusriiki ja demokraatiat tugevdavaid reforme...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, -0.000396728515625, -0.00...",16000


In [109]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,esiteks on oluline et euroopa liit tuleks välj...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005340576171875, -0.004791259765625, -0.00...",16000
1,male,härra eesistuja juba süüria kodusõda tekitas s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001129150390625, -0.001190185546875, -0.00...",16000
2,male,ja lõpetuseks tuleb kehtestada rahvusvahelised...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.014007568359375, 0.0140380859375, 0.0124206...",16000
9,male,õigusriiki ja demokraatiat tugevdavaid reforme...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, -0.000396728515625, -0.00...",16000
10,male,samas on mitme idapartnerluse riigi puhul jätk...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002593994140625, 0.00177001953125, -0.0029...",16000
11,male,ja energialiiduna võib ühel päeval olla palju ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 3.0517578125e-05, -6.103515625e-05, -9.1...",16000
12,male,me oleme olnud liiga palju üllatunud venemaa k...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00152587890625, -0.00250244140625, -0.0009...",16000
14,male,põgenikud nende abistamine peab olema euroopa ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, -3.0517578125e-05, 6.10351...",16000
19,male,novembrikuisel idapartnerluse tippkohtumisel t...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005096435546875, -0.000213623046875, 0.001...",16000
22,male,kuidas te hindate viimasel ajal serbia mõju om...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, 0.028228759765625, -0.011...",16000


In [110]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
3,female,eelnimetatud rahvusvahelise õiguse rikkumised ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001190185546875, -0.00128173828125, -0.000...",16000
4,female,mulle meeldib väga punkt üks kus öeldakse et i...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.07452392578125, 0.05059814453125, 0.0300292...",16000
5,female,ja ma loeksin ette hea meelega resolutsiooni p...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.038238525390625, 0.0269775390625, 0.0199890...",16000
6,female,siin on suur tähtsus mitteformaalse ja informa...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001983642578125, -0.0028076171875, -0.00152...",16000
7,female,elu kinnitas taas kord et ida eurooplaste kann...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.017974853515625, -0.020294189453125, -0.018...",16000
8,female,euroopa liidule ja eriti rahva poolt otse vali...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005035400390625, 0.001007080078125, 0.0037...",16000
13,female,muidugi ma eeldan seda et ukraina jätkab oma r...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00726318359375, 0.0023193359375, 0.0048522...",16000
15,female,meie kohus on näidata et selline käitumine ei ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, -0.0003662109375, -0.00317...",16000
16,female,kuid globaalne majanduskriis õõnestas meie usk...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005279541015625, -0.0052490234375, -0.0061...",16000
17,female,kahjuks andis eilsel väliskomisjoni ja moldova...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001861572265625, 0.00946044921875, 0.004943...",16000


In [111]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="et", fp16=False)
    return result["text"]

In [112]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [113]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [114]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.8076923076923077, Precision: 0.46153846153846156, Recall: 0.48, F1: 0.47058823529411764
Processed 2 files, WER: 0.8571428571428571, Precision: 0.3181818181818182, Recall: 0.35, F1: 0.3333333333333333
Processed 3 files, WER: 0.6, Precision: 0.5, Recall: 0.4, F1: 0.4444444444444445
Processed 4 files, WER: 0.5333333333333333, Precision: 0.5, Recall: 0.5333333333333333, F1: 0.5161290322580646
Processed 5 files, WER: 0.8571428571428571, Precision: 0.2857142857142857, Recall: 0.2857142857142857, F1: 0.2857142857142857
Processed 6 files, WER: 0.5588235294117647, Precision: 0.5, Recall: 0.4838709677419355, F1: 0.4918032786885246
Processed 7 files, WER: 0.52, Precision: 0.4782608695652174, Recall: 0.4782608695652174, F1: 0.4782608695652174
Processed 8 files, WER: 0.6666666666666666, Precision: 0.42857142857142855, Recall: 0.3333333333333333, F1: 0.375
Processed 9 files, WER: 0.9615384615384616, Precision: 0.20689655172413793, Recall: 0.23076923076923078, F1: 0.21818181

In [115]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.6842105263157895, Precision: 0.3125, Recall: 0.2777777777777778, F1: 0.29411764705882354
Processed 2 files, WER: 0.782608695652174, Precision: 0.375, Recall: 0.2727272727272727, F1: 0.3157894736842105
Processed 3 files, WER: 0.6363636363636364, Precision: 0.42105263157894735, Recall: 0.38095238095238093, F1: 0.4
Processed 4 files, WER: 0.6, Precision: 0.35714285714285715, Recall: 0.35714285714285715, F1: 0.35714285714285715
Processed 5 files, WER: 0.5833333333333334, Precision: 0.43478260869565216, Recall: 0.4166666666666667, F1: 0.425531914893617
Processed 6 files, WER: 0.6470588235294118, Precision: 0.4, Recall: 0.35294117647058826, F1: 0.37500000000000006
Processed 7 files, WER: 0.6363636363636364, Precision: 0.35135135135135137, Recall: 0.34210526315789475, F1: 0.3466666666666667
Processed 8 files, WER: 0.5, Precision: 0.5333333333333333, Recall: 0.5714285714285714, F1: 0.5517241379310344
Processed 9 files, WER: 0.8125, Precision: 0.23529411764705882, Reca